### Add Peer Grouping To Windows Host Data

This notebook introduces peer grouping - the idea that we can segment a network into smaller groups of similar devices / users / etc. This segmentation permits finer resolution of outliers.

In this example, we break the notebook into two peer groups based on whether it is a `User` or a `Computer` account. Users correspond to actual users, computers to system accounts.

__Damian Herrick__  
__SAS Institute__  
__[damian.herrick@sas.com](mailto:damian.herrick@sas.com)__  

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import swat
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

Standard getting started stuff:
* set environment variables.
* establish CAS connection.
* set active CASLIB.
* load our dataset.

In [44]:
os.environ["CAS_CLIENT_SSL_CA_LIST"]="/home/ds/cascert.pem"

conn = swat.CAS("d-crd-datasci01big.dev.cyber.sas.com", 5570)

conn.setsessopt('WH')

out = conn.loadtable('wls_day_02_hr13.sashdat', caslib='WH')

NOTE: Cloud Analytic Services made the file wls_day_02_hr13.sashdat available as table WLS_DAY_02_HR13 in caslib CASUSER(daherr).


This time, we'll add a column using datastep code.

We'll rely on the conditional functionality to determine a peer group type.

In [45]:
sasCode = F'''
    data wls_day_02_hr13_pg;
        set wls_day_02_hr13;
        if find(UserName, '$', 'i') then peerGroup = "Computers";
        else if find(UserName, 'User') then peerGroup = "Users";
        else peerGroup = "Computers";
'''

print(sasCode)


    data wls_day_02_hr13_pg;
        set wls_day_02_hr13;
        if find(UserName, '$', 'i') then peerGroup = "Computers";
        else if find(UserName, 'User') then peerGroup = "Users";
        else peerGroup = "Computers";



In [47]:
conn.datastep.runcode(sasCode)
conn.promote(name='wls_day_02_hr13_pg')

ERROR: The target table wls_day_02_hr13_pg of the promotion already exists. Please specify a different name.
ERROR: The action stopped due to errors.


+ Elapsed: 0.0228s, user: 0.00406s, sys: 0.0185s, mem: 0.265mb

In [48]:
conn.fetch(table={'name':'wls_day_02_hr13_pg'}, maxrows=20)

,AuthenticationPackage,Destination,DomainName,EventID,FailureReason,LogHost,LogonID,LogonType,LogonTypeDescription,ParentProcessID,...,ServiceName,Source,Status,SubjectDomainName,SubjectLogonID,SubjectUserName,Time,UserName,occurredTime,peerGroup
0,,,Domain001,4688.0,,Comp755918,0x3e7,NaN,,0x1750,...,,,,,,,135368.0,Comp755918$,13:36:08,Computers
1,NTLM,,EnterpriseAppServer,4624.0,,EnterpriseAppServer,0x191d22f1,3.0,Network,,...,,EnterpriseAppServer,,,,,135368.0,AppService,13:36:08,Computers
2,MICROSOFT_AUTHENTICATION_PACKAGE_V1_0,,Domain001,4776.0,,ActiveDirectory,,NaN,,,...,,Comp935081,0x0,,,,135368.0,Administrator,13:36:08,Computers
3,,,Domain001,4688.0,,Comp827750,0x3e7,NaN,,0x360,...,,,,,,,135368.0,Comp827750$,13:36:08,Computers
4,,,Domain001,4688.0,,Comp755918,0x3e7,NaN,,0x1750,...,,,,,,,135368.0,Comp755918$,13:36:08,Computers
5,,,Domain001,4688.0,,Comp755918,0x3e7,NaN,,0x237c,...,,,,,,,135368.0,Comp755918$,13:36:08,Computers
6,Kerberos,,Domain001,4624.0,,Comp916004,0x22bdef171,3.0,Network,,...,,Comp916004,,,,,135368.0,Comp916004$,13:36:08,Computers
7,,,Domain001,4672.0,,Comp916004,0x22bdef67d,NaN,,,...,,,,,,,135368.0,Comp916004$,13:36:08,Computers
8,Kerberos,,Domain001,4624.0,,ActiveDirectory,0x37854502,3.0,Network,,...,,Comp077821,,,,,135368.0,User337163,13:36:08,Users
9,Kerberos,,Domain001,4624.0,,EnterpriseAppServer,0x9f84da7,3.0,Network,,...,,EnterpriseAppServer,,,,,135368.0,EnterpriseAppServer$,13:36:08,Computers


In [49]:
conn.table.save(table='wls_day_02_hr13_pg', name='wls_day_02_hr13_pg.sashdat', caslib='WH', replace=True)

NOTE: Cloud Analytic Services saved the file wls_day_02_hr13_pg.sashdat in caslib WH.


[caslib]

 'WH'

[name]

 'wls_day_02_hr13_pg.sashdat'

+ Elapsed: 0.439s, user: 0.0429s, sys: 0.375s, mem: 16.4mb

In [50]:
conn.close()